In [33]:
from PIL import Image
import numpy as np
import pickle

In [34]:
im = Image.open('./kentuk.nosync/slope_sm_res10.tif')
slopearray = np.array(im)
print(slopearray.shape)

(654, 1155)


In [35]:
import rioxarray

rds = rioxarray.open_rasterio('./kentuk.nosync/slope_sm_res10.tif')
df = rds.to_dataframe(name="slope")

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import shapefile

polygons = []
sf = shapefile.Reader("Reference data/slope_deformations_reference_polygons/reference_data_existing_landslides_polygons.shp")
shapes = sf.shapes()
rectangles = []

for shape in shapes:
    polygons.append(Polygon(shape.points))
    rectangles.append(shape.bbox)

is_landslide = set()
for i in range(len(polygons)):
    rectangle = rectangles[i]
    low_x, low_y, high_x, high_y = rectangle[0], rectangle[1], rectangle[2], rectangle[3]
    small_df = df.query('y >= @low_y and y <= @high_y and x >= @low_x and x <= @high_x')
    if(len(small_df) > 0):
        for _, y, x in small_df.index:
            if polygons[i].contains(Point(x, y)):
                is_landslide.add((x, y))

bools = np.array([True if (x, y) in is_landslide else False for _, y, x in df.index])
df.insert(loc = 0, column='is_in', value=bools)
rds = df.reset_index().drop(columns=['band', 'spatial_ref'])

In [36]:
def preprocess_numpy_array(coordinates, original):
    result = []
    for i in range(original.shape[0]):
        result.append([])
        for j in range(original.shape[1]):
            current = coordinates.loc[i * original.shape[0] + j]
            # result[i].append(current['is_in_polygon'])
            result[i].append(1 if current['is_in'] else 0)

    return np.array(result)

In [37]:
X_test = list()
Y_test = list()
try:
    with open('10smtest.pickle', 'rb') as handle:
        X_test, Y_test = pickle.load(handle)
except:
    np_coords: np.ndarray = preprocess_numpy_array(rds, slopearray)
    for i in range(0, slopearray.shape[0] - 100,30):
        for j in range(0, slopearray.shape[1] - 100, 30):
            current_array: np.ndarray = slopearray[i:i+100, j:j+100].flatten()
            X_test.append(current_array)
            Y_test.append(np.mean(np_coords[i:i+100, j:j+100]))
    
    with open('10smtest.pickle', 'wb') as handle:
        pickle.dump((X_test, Y_test), handle)

In [38]:
from sklearn.linear_model import LinearRegression

x = np.array(X_test)
y = np.array(Y_test)

model = LinearRegression().fit(x, y)

In [43]:
whole_slopes = np.array(Image.open('./kentuk.nosync/slope_whole_res10.tif'))
result = []
for i in range(0, whole_slopes.shape[0] - 100,30):
    for j in range(0, whole_slopes.shape[1] - 100, 30):
        current_array: np.ndarray = whole_slopes[i:i+100, j:j+100].flatten()
        result.append(model.predict(np.array([current_array])))

result = np.array(result).resize(whole_slopes.)
print(result)

0.19861549
